In [450]:
import math
import hashlib
import random
import numpy as np
import string

In [451]:
def get_password(student_username, l=10):
    # possible characters include upper-case English letters,
    # numbers between 0 and 9 (inclusive),
    # and the underscore symbol
    options = string.digits + string.ascii_uppercase + "_"
        
    h = hashlib.sha256(("ECS759P-AI"+student_username).encode("utf-8"))
    d = h.digest()
    s = ""
    for n in d:
        s += options[n%len(options)]
    return s[0:l]

In [452]:
# hyperparameters

POPULATION_SIZE = 500
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.9
TARGET_PASSWORD = get_password('ec23759') # "TRT9_2U6R8"
GENERATIONS = 1000

In [453]:
# random password generator
def generate_random_password(length):
    return ''.join(random.choice(string.ascii_uppercase + string.digits + '_') for _ in range(length))

In [454]:
def distance_function(string_one, string_two):
    score = 0
    for i, j in zip(string_one, string_two):
        score += math.sqrt(abs(ord(i) - ord(j)))
    return score

In [455]:
MAX_VALUE = distance_function('0000000000', '__________')

In [456]:
def get_normalised_fitness(list_of_phrases, phrase_to_find):
    ordered_dict = dict()
    for phrase in list_of_phrases:
        ordered_dict[phrase] = 1 - (distance_function(phrase, phrase_to_find) / MAX_VALUE)
    return ordered_dict

In [457]:
# multi-point crossover directed by the corssover rate
def crossover(parent1, parent2, crossover_rate):
    if np.random.rand() <= crossover_rate:
        crossover_point1 = random.randint(0, len(parent1))
        crossover_point2 = random.randint(crossover_point1, len(parent1))
        child1 = parent1[:crossover_point1] + parent2[crossover_point1:crossover_point2] + parent1[crossover_point2:]
        child2 = parent2[:crossover_point1] + parent1[crossover_point1:crossover_point2] + parent2[crossover_point2:]
        return child1, child2
    return parent1, parent2

In [458]:
# mutation
def mutate(password, mutation_rate):
    mutated_password = list(password)
    m_len = len(mutated_password)
    for i in range(m_len):
        if random.random() <= mutation_rate:
            mutated_password[i] = random.choice(string.ascii_uppercase + string.digits + '_')
    return ''.join(mutated_password)

In [459]:
def genetic_algorithm(target_password, population_size, mutation_rate, crossover_rate):
    
    population = [generate_random_password(len(target_password)) for _ in range(population_size)]

    for generation in range(GENERATIONS):
        fitness_scores = get_normalised_fitness(population, target_password)

        # check for convergence ie. true password was found
        if max(fitness_scores.values()) == 1.:
            solution = max(fitness_scores, key=fitness_scores.get)
            return generation, solution
        
        # sort through the fitness values of population
        sorted_population = [k for k, _ in sorted(fitness_scores.items(), key=lambda item: item[1], reverse=True)]
        # selection criteria: eliminate the half of candidates with less fitness scores
        selected_population = sorted_population[:len(sorted_population)//2]

        new_population = selected_population.copy()

        while len(new_population) < population_size:
            # parent selection at random
            parent1 = random.choice(selected_population)
            parent2 = random.choice(selected_population)

            # crossover
            child1, child2 = crossover(parent1, parent2, crossover_rate)

            # check if the offspring from crossover is same as the parent to restart the parent selection process
            if (child1, child2) == (parent1, parent2):
                continue
                
            # mutation
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            # extend the population by adding the offsprings for the next generation
            new_population.extend([child1, child2])

        population = new_population

In [460]:
genetic_algorithm(TARGET_PASSWORD, POPULATION_SIZE, MUTATION_RATE, CROSSOVER_RATE)

(29, 'TRT9_2U6R8')

In [461]:
experiments = [genetic_algorithm(TARGET_PASSWORD, POPULATION_SIZE, MUTATION_RATE, CROSSOVER_RATE)[0] for _ in range(10)]
print(f"Generation of convergence: {experiments}, Mean convergence: {np.mean(experiments)},  std of convergence: {np.std(experiments)}")

Generation of convergence: [24, 29, 20, 25, 22, 28, 25, 27, 23, 27], Mean convergence: 25.0,  std of convergence: 2.6832815729997477
